# Fine tuning T5 base arabic to perform Arabic abstractive summarization

In [5]:
%pip install transformers[torch] evaluate datasets rouge-score nltk
%pip install accelerate -U

  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached rouge_score-0.1.2.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=8d885b082d6850fdaf0b2d3bcd8949ec48fec104849ccc07ee729eb4518f7272
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 8.3 MB/s eta 0:00:00:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.33.0
    Uninstalling accelerate-0.33.0:
      Successfully uninstalled accelerate-0.33.0
Note: you may need to restart the kernel to use updated packages.


In [6]:
!nvidia-smi

Wed Sep 18 05:02:09 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.07              Driver Version: 550.90.07      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P0             28W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%pip install accelerate

In [7]:
%pip install arabert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.1 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186459 sha256=33d0cde14c88e29ffe18fefbf026779cd01b94e02cda4fab0ef89d5149ad88be
  Stored in directory: /root/.cache/pip/wheels/10/f0/fd/4813b1177405693e8da9cdea839f0fb64fde161380e058c827
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.12.1
    Uninstalling emoji-2.12.1:
      Successfully uninstalled emoji-2.12.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install datasets

In [ ]:
%pip install evaluate

In [ ]:
%pip install rouge_score

In [8]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Loading the dataset
Loading the **AGS-Corpus** with arabic articles and their summaries from *HuggingFace*

In [9]:
import torch
from datasets import load_dataset

ds = load_dataset("FahdSeddik/AGS-Corpus", split=["train"])

Generating train split:   0%|          | 0/141467 [00:00<?, ? examples/s]

Getting the dataset

In [10]:
train_ds = ds[0]
train_ds

Dataset({
    features: ['id', 'text', 'summary'],
    num_rows: 141467
})

Now that we've loaded the dataset from HuggingFace we can look at an example

In [11]:
example = train_ds[0]
article = example['text']
summary = example['summary']
print("Article\n", article)
print("Summary\n", summary)

Article
 يمكن تصور السلوكيات المُهَدِدة باعتبارها ثمرة لعدم القدرة على التأقلم مع الدافع الطبيعي التنافسي المتعلق بعلاقات الهيمنة المتبادلة التي تلاحظ عامة بين الحيوانات. بدلاً من ذلك، قد ينتج الترهيب في مجتمع من نوع يكون أفراده اجتماعيين، فالبشر بشكل عام يترددون في الدخول في مواجهة أو تهديد عنيف.وهو سلوك مثله مثل جميع السمات السلوكية يظهر بشكل أزيد أو أقل في كل فرد مع مرور الزمن، ولكنه قد يكون «سلوك تعويضي» ذو أهمية كبيرة بالنسبة للبعض مقارنة بالآخرين. فإن المنظرين السلوكيين كثيراً ما يرون أن السلوكيات المُهَدِدة هي نتيجة لتعرض القائمين بها للتهديد من قبل الآخرين، بما في ذلك الآباء، ورموز السلطة، والرفاق والأشقاء. «استخدام القوة مبرر عندما يعتقد الشخص بشكل منطقي أنها ضرورية للدفاع عن النفس أو الآخرين تجاه الاستخدام الفوري لقوة غير شرعية».و قد يتم استخدام الترهيب بوعي أو بغير وعي، ونسبة من الأشخاص الذين يستخدمونه بوعي ربما يفعلون ذلك نتيجة أفكار مستوعبة بأنانية عن تخصيصه لغرض، أو لفائدة أو للتمكين الذاتي. الترهيب المتصل بالتحامل والتمييز يمكن أن يشمل السلوك «الذي يزعج، يهدد، يرهب، وينذ

## Encoding the dataset
Encoding the dataset using the tokenizer and add the "Task prefix" as the T5 model needs the summarization prefix
  
### Preprocessing using the `ArabertPreprocessor`
The ArabertPreprocessor is designed to normalize Arabic text for better performance when using Arabic NLP models. Specifically, it does things like:

* **Removing diacritics** (which are optional in Arabic but can add noise for models not trained with them).
* **Normalizing different forms of Arabic letters** (like "أ", "إ", "آ" being reduced to "ا").
* **Handling Arabic-specific punctuation and word forms** that may affect tokenization   

In [12]:
# import the tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# import preprocessor
from arabert.preprocess import ArabertPreprocessor

# initialize the tokenizer from the model
tokenizer = AutoTokenizer.from_pretrained("malmarjeh/t5-arabic-text-summarization")
# intialize preprocessor
preprocessor = ArabertPreprocessor(model_name = "")

prefix = 'تلخيص: '
max_input_length = 512
max_output_length = 200

def preprocess_examples(examples):
  # preprocess using the arabertpreprocessor
  articles = [preprocessor.preprocess(article) for article in examples['text']]
  summaries = [preprocessor.preprocess(summary) for summary in examples['summary']]

  # add the prefix to the input
  inputs = [prefix + article for article in articles]
  # tokenize inputs
  tokenized_inputs =  tokenizer(inputs, max_length = max_input_length, padding= "max_length", truncation = True)
  # encode the summaries
  labels = tokenizer(summaries, max_length=max_output_length, padding="max_length", truncation=True).input_ids

  # important: we need to replace the index of the padding tokens by -100
  # such that they are not taken into account by the CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)

  tokenized_inputs["labels"] = labels_with_ignore_index

  return tokenized_inputs

tokenizer_config.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
# encode the dataset
encoded_dataset =  train_ds.map(preprocess_examples, batched=True, remove_columns=train_ds.column_names)

Map:   0%|          | 0/141467 [00:00<?, ? examples/s]

verifying an example by decoding back to text

In [14]:
labels = encoded_dataset[0]['labels']
print(labels)
tokenizer.decode([x for x in labels if x != -100])

[58335, 83393, 345, 24, 268, 1280, 493, 4727, 15, 79492, 46, 1090, 39826, 30219, 30471, 87, 9622, 3, 4, 140, 15303, 60, 47587, 8, 4519, 6, 4279, 2387, 3, 4, 60, 47587, 520, 30365, 71963, 82, 3080, 8778, 3, 5, 41199, 2627, 194, 26477, 13, 71963, 1586, 23075, 133, 408, 9257, 24, 5462, 3, 4, 60, 47587, 345, 24, 8230, 64421, 51593, 3, 5, 82, 30327, 63222, 3, 5, 82, 19926, 88261, 28, 3, 5, 82, 60, 44750, 67818, 14, 50, 82, 8264, 62642, 3, 4, 140, 4454, 9648, 2445, 5038, 33398, 3, 4, 1, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100

'السلوكيات المهددة يمكن أن تكون نتيجة عدم القدرة على التأقلم مع العلاقات الهيمنة التنافسية المتبادلة بين الحيوانات. قد ينتج الترهيب في مجتمع من النوع الاجتماعي. الترهيب يتم استخدامه بوعي أو بغير وعي ، ونسبة الأشخاص الذين يستخدمونه بوعي ربما يفعلون ذلك بسبب أفكار أنانية. الترهيب يمكن أن يشمل الاحتكاك البدني ، أو التلاعب بالمشاعر ، أو الإساءة اللفظية ، أو الإحراج المتعمد و - أو الاعتداء الجسدي. قد يشكل تهديد العنف جريمة جنائية.</s>'

Now we set the format to PyTorch

In [15]:
encoded_dataset.set_format(type = 'torch')

split the dataset to train/test sets

In [16]:
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

def split_and_create_loaders(dataset, train_batch_size = 8, val_batch_size=16, test_batch_size=16):
    # define train and test size
    train_size = int(0.8*len(dataset))
    val_size = int(0.1*len(dataset))
    test_size = len(dataset)- (val_size + train_size)

    # split using the random_split function
    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])
    # create a dataloader for each set
    train_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size = val_batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size = test_batch_size, shuffle= True)

    return train_loader, val_loader, test_loader

In [17]:
train_loader, val_dataset, test_loader = split_and_create_loaders(encoded_dataset)

## Fine-tuning the model

`Seq2SeqTrainer` and `Seq2SeqTrainingArguments` inherit from the `Trainer` and `TrainingArgument` classes and they’re adapted for training models for sequence-to-sequence tasks such as summarization or translation. Which is why we will use those in the fine-tuning process

In [18]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq, AdamW, AutoModelForSeq2SeqLM
import torch

torch.cuda.empty_cache()
# load the model
model = AutoModelForSeq2SeqLM.from_pretrained("malmarjeh/t5-arabic-text-summarization")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move the model to the appropriate device

# define training arguments object
training_arguments = Seq2SeqTrainingArguments(
    output_dir = "./results",  # output directory
    eval_strategy = "epoch",  # evaluating every epoch
    learning_rate = 2e-5,
    per_device_train_batch_size = 8,  # batch size for training
    per_device_eval_batch_size = 16,  # batch size for evaluation
    weight_decay = 0.01,
    save_total_limit = 2,  # only keep the last 2 models
    num_train_epochs = 1,  # nbr of  training epochs
    predict_with_generate = True,  # enable text generation for text evaluation
    fp16=True,
    report_to="none",  # Disables Weights & Biases
)

# initialize data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

pytorch_model.bin:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

### This run's hyperparameters:
`epochs` 1  
`learning rate` 0.00002  
`max input length` 512  
`max output length` 200 
In the next run we want to see how it will handle summarization of longer sequences into short paragraphs

In [19]:
# initialize trainer
trainer = Seq2SeqTrainer(
    model,
    training_arguments,
    train_dataset = train_loader.dataset,
    eval_dataset = test_loader.dataset,
    data_collator = data_collator,
    tokenizer = tokenizer,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/opt/conda/lib/python3.10/site-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss
1,2.659500,2.204502


TrainOutput(global_step=14147, training_loss=2.8351976272368367, metrics={'train_runtime': 15198.1278, 'train_samples_per_second': 7.447, 'train_steps_per_second': 0.931, 'total_flos': 6.891759134834688e+16, 'train_loss': 2.8351976272368367, 'epoch': 1.0})

In [20]:
eval_results = trainer.evaluate()

In [21]:
eval_results

{'eval_loss': 2.2045016288757324,
 'eval_runtime': 584.1544,
 'eval_samples_per_second': 24.22,
 'eval_steps_per_second': 1.515,
 'epoch': 1.0}

In [27]:
# save the fine-tuned model
trainer.save_model("/kaggle/outputs/fine_tuned_model")
tokenizer.save_pretrained("/kaggle/outputs/fine_tuned_model")

('/kaggle/outputs/fine_tuned_model/tokenizer_config.json',
 '/kaggle/outputs/fine_tuned_model/special_tokens_map.json',
 '/kaggle/outputs/fine_tuned_model/spiece.model',
 '/kaggle/outputs/fine_tuned_model/added_tokens.json',
 '/kaggle/outputs/fine_tuned_model/tokenizer.json')

In [38]:
# save the fine-tuned model
trainer.save_model("./model")
tokenizer.save_pretrained("./model")

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/spiece.model',
 './model/added_tokens.json',
 './model/tokenizer.json')

In [36]:
import shutil
import zipfile
import os
from IPython.display import FileLink

# Path to the folder you want to zip
folder_path = '/kaggle/working/fine_tuned_model'
# Path where you want to save the zip file
zip_file_path = '/kaggle/working/fine_tuned_model.zip'

# Create a zip file of the folder
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', folder_path)

print(f"Model saved and zipped at: {zip_file_path}")

FileLink('/kaggle/working/fine_tuned_model.zip')

Model saved and zipped at: /kaggle/working/fine_tuned_model.zip


/kaggle/working/fine_tuned_model.zip

In [45]:
FileLink(r'fine_tuned_model.zip')

/kaggle/working/fine_tuned_model.zip

### Testing on an external aritcle

In [43]:
# Load the fine-tuned model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/outputs/fine_tuned_model")
tokenizer = AutoTokenizer.from_pretrained("/kaggle/outputs/fine_tuned_model")

# Example external text to summarize
external_text = "قدمت ملك ريان مذيعة موقع صدي البلد تغطية خاصة حول وقف خطة تخفيف الأحمال  حيث انتهت أمس المدة المقررة لوقف خطة تخفيف الأحمال والجميع أصبح يتساءل عن موعد عودة العمل بخطة تخفيف الأحمال من جديد فهل بالفعل سيتم قطع الكهرباء مرة أخرى؟ أم سيتم تجديد فترة وقف قطع الكهرباء؟ وما هي القرارات الجديدة المنتظر إعلانها بهذا الشأن؟دعونا نتابع معكم كل التفاصيل الخاصة بما سيحدث بشأن خطة تخفيف الأحمال خلال الأيام المقبلة من خلال هذه التغطية.انتهت أمس المدة التي حددها مجلس الوزراء برئاسة الدكتور مصطفى مدبولي، لوقف خطة تخفيف الأحمال، حيث كان من المقرر أن تكون مدة إيقاف قطع الكهرباء وتعليق خطة تخفيف الأحمال تكون من 21 يوليو الماضي وتستمر حتى يوم أمس الأحد 15 سبتمبر. ولذلك يترقب المصريون القرار الجديد الذي ستتخذه الحكومة حول مصير خطة تخفيف أحمال الكهرباء، واصبح البحث عن موعد عودة تخفيف أحمال الكهرباء متصدر محرك البحث جوجل خلال الساعات الماضية.ولكن حتى هذه اللحظة لم تصدر الحكومة أو وزارتا الكهرباء والطاقة المتجددة والبترول والثروة المعدنية أية بيانات رسمية، حتى الآن، تؤكد عودة تخفيف الأحمال وفي إطار جهودها لتأمين احتياجات شبكة الكهرباء وتجنب عودة تخفيف الأحمال، أبرمت وزارة البترول والثروة المعدنية عقودًا لشحنات جديدة من الغاز المسال كما أكد مصدر مسؤول في قطاع الكهرباء أنه لم يصل أي قرار إلى شركات الكهرباء بشأن العودة إلى تنفيذ خطة تخفيف الأحمال قبل تعليقها في 21 يوليو الماضي ومن جانبه، صرح حمدي عبد العزيز، المتحدث الرسمي لوزارة البترول والثروة المعدنية بأن عدد الشحنات المتعاقد عليها زاد من 21 شحنة إلى 32 شحنة، في إطار جهود ضمان استمرار تدفق الغاز إلى محطات توليد الكهرباء، وأشار إلى أن الغاز المسال المستورد يتم تحويله إلى غاز طبيعي وضخه في الشبكة القومية للكهرباء عبر منشآت متخصصة، مثل السفينة الموجودة في شرم الشيخ، التي تقوم بعملية التحويل وضخ الغاز بشكل مستمر لضمان استقرار إمدادات الكهرباء.وبالنسبة لإمكانية العودة لتخفيف أحمال الكهرباء قريبا، أكد عبد العزيز أنه من الصعب تأكيد أو نفي هذا الاحتمال حاليًا، وطمأن المصريين بأنه لا يوجد نقص في هذه الإمدادات الضرورية لتوليد الكهرباء."

# Preprocess the text (tokenize)
inputs = tokenizer(external_text, return_tensors="pt", max_length=512, truncation=True).input_ids

# Generate the summary using the model
summary_ids = model.generate(inputs, max_length=150, num_beams=5, length_penalty=1.2, early_stopping=True)

# Decode the generated summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the summary
print("Generated Summary:\n", summary)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Generated Summary:
 أعلنت وزارة البترول والطاقة المتجددة والبترول والثروة المعدنية عن وقف خطة تخفيف الأحمال بعد انتهاء المدة المقررة لوقفها في 21 يوليو الماضي ، وتم توقيع عقود لشحنات جديدة من الغاز المسال ، وتم تحويل الغاز المسال إلى غاز طبيعي وضخه في الشبكة القومية للكهرباء. لم تصدر الحكومة أو وزارات الكهرباء والطاقة المتجددة والبترول والثروة المعدنية أي بيانات رسمية تؤكد عودة تخفيف الأحمال. لم يصدر أي قرار بشأن العودة إلى تنفيذ خطة تخفيف الأحمال بعد تعليقها في 21 يوليو الماضي.


#### Original Article:
قدمت ملك ريان مذيعة موقع صدي البلد تغطية خاصة حول وقف خطة تخفيف الأحمال  حيث انتهت أمس المدة المقررة لوقف خطة تخفيف الأحمال والجميع أصبح يتساءل عن موعد عودة العمل بخطة تخفيف الأحمال من جديد فهل بالفعل سيتم قطع الكهرباء مرة أخرى؟ أم سيتم تجديد فترة وقف قطع الكهرباء؟ وما هي القرارات الجديدة المنتظر إعلانها بهذا الشأن؟
دعونا نتابع معكم كل التفاصيل الخاصة بما سيحدث بشأن خطة تخفيف الأحمال خلال الأيام المقبلة من خلال هذه التغطية.

انتهت أمس المدة التي حددها مجلس الوزراء برئاسة الدكتور مصطفى مدبولي، لوقف خطة تخفيف الأحمال، حيث كان من المقرر أن تكون مدة إيقاف قطع الكهرباء وتعليق خطة تخفيف الأحمال تكون من 21 يوليو الماضي وتستمر حتى يوم أمس الأحد 15 سبتمبر. 
ولذلك يترقب المصريون القرار الجديد الذي ستتخذه الحكومة حول مصير خطة تخفيف أحمال الكهرباء، واصبح البحث عن موعد عودة تخفيف أحمال الكهرباء متصدر محرك البحث جوجل خلال الساعات الماضية.

ولكن حتى هذه اللحظة لم تصدر الحكومة أو وزارتا الكهرباء والطاقة المتجددة والبترول والثروة المعدنية أية بيانات رسمية، حتى الآن، تؤكد عودة تخفيف الأحمال.
وفي إطار جهودها لتأمين احتياجات شبكة الكهرباء وتجنب عودة تخفيف الأحمال، أبرمت وزارة البترول والثروة المعدنية عقودًا لشحنات جديدة من الغاز المسال. 
كما أكد مصدر مسؤول في قطاع الكهرباء أنه لم يصل أي قرار إلى شركات الكهرباء بشأن العودة إلى تنفيذ خطة تخفيف الأحمال قبل تعليقها في 21 يوليو الماضي.

ومن جانبه، صرح حمدي عبد العزيز، المتحدث الرسمي لوزارة البترول والثروة المعدنية بأن عدد الشحنات المتعاقد عليها زاد من 21 شحنة إلى 32 شحنة، في إطار جهود ضمان استمرار تدفق الغاز إلى محطات توليد الكهرباء، وأشار إلى أن الغاز المسال المستورد يتم تحويله إلى غاز طبيعي وضخه في الشبكة القومية للكهرباء عبر منشآت متخصصة، مثل السفينة الموجودة في شرم الشيخ، التي تقوم بعملية التحويل وضخ الغاز بشكل مستمر لضمان استقرار إمدادات الكهرباء.

وبالنسبة لإمكانية العودة لتخفيف أحمال الكهرباء قريبا، أكد عبد العزيز أنه من الصعب تأكيد أو نفي هذا الاحتمال حاليًا، وطمأن المصريين بأنه لا يوجد نقص في هذه الإمدادات الضرورية لتوليد الكهرباء.

##### Generated Summary
 أعلنت وزارة البترول والطاقة المتجددة والبترول والثروة المعدنية عن وقف خطة تخفيف الأحمال بعد انتهاء المدة المقررة لوقفها في 21 يوليو الماضي وتستمر حتى 15 سبتمبر. تم توقيع عقود لشحنات جديدة من الغاز المسال ، ولكن لم يصل أي قرار إلى شركات الكهرباء بشأن العودة إلى تنفيذ خطة تخفيف الأحمال قبل تعليقها في 21 يوليو الماضي. يتم تحويل الغاز المسال إلى غاز طبيعي وضخه في الشبكة القومية للكهرباء. لا يوجد نقص في الإمدادات الضرورية لتوليد الكهرباء.

In [79]:
# Load the fine-tuned model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/outputs/fine_tuned_model")
tokenizer = AutoTokenizer.from_pretrained("/kaggle/outputs/fine_tuned_model")

# Example external text to summarize
external_text = "لقد أصبح الذكاء الاصطناعي مصطلحًا شاملًا للتطبيقات التي تؤدي مهام مُعقدة كانت تتطلب في الماضي إدخالات بشرية مثل التواصل مع العملاء عبر الإنترنت أو ممارسة لعبة الشطرنج. يُستخدم غالبًا هذا المصطلح بالتبادل مع مجالاته الفرعية، والتي تشمل التعلم الآلي (ML) والتعلم العميق.ومع ذلك، هناك اختلافات.. على سبيل المثال، يُركز التعلم الآلي على إنشاء أنظمة تتعلم أو تحسّن من أدائها استنادًا إلى البيانات التي تستهلكها. ومن المهم أن نلاحظ أنه على الرغم من أن كل سُبل التعلم الآلي ما هي إلّا ذكاء اصطناعي، فإنه ليس كل ذكاء اصطناعي يُعد تعلمًا آليًا.للحصول على القيمة الكاملة من الذكاء الاصطناعي، تقوم العديد من الشركات باستثمارات كبيرة في فرق علوم البيانات. يجمع علم البيانات بين الإحصاءات وعلوم الكمبيوتر والمعرفة بالأعمال لاستخلاص القيمة من مصادر البيانات المختلفة. يستخدم المطورون الذكاء الاصطناعي لأداء المهام التي يتم تنفيذها يدويًا بكفاءة أكبر، والتواصل مع العملاء، وتحديد الأنماط، وحل المشكلات. للبدء في استخدام الذكاء الاصطناعي، يجب أن يكون للمطورين خلفية في الرياضيات ويشعرون بالراحة مع الخوارزميات.عند البدء باستخدام الذكاء الاصطناعي لإنشاء تطبيق، يساعد على البدء على نطاق صغير. من خلال بناء مشروع بسيط نسبيًا، مثل على سبيل المثال، ستتعلم أساسيات الذكاء الاصطناعي. يعد التعلم عن طريق الممارسة وسيلة رائعة لتحسين أي مهارة، والذكاء الاصطناعي لا يختلف عن ذلك. بمجرد الانتهاء من مشروع صغير أو أكثر بنجاح، لا توجد حدود للمكان الذي يمكن أن يأخذك فيه الذكاء الاصطناعي."
# Preprocess the text (tokenize)
inputs = tokenizer(external_text, return_tensors="pt", max_length=512, truncation=True).input_ids

# Generate the summary using the model
summary_ids = model.generate(inputs, max_length=150, num_beams=5, length_penalty=1.2, early_stopping=True)

# Decode the generated summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the summary
print("Generated Summary:\n", summary)

Generated Summary:
 الذكاء الاصطناعي مصطلح شامل للتطبيقات التي تؤدي مهام مُعقدة كانت تتطلب إدخالات بشرية. يمكن استخدام هذا المصطلح بالتبادل مع مجالاته الفرعية وتشمل التعلم الآلي والتعلم العميق والتعلم العميق. يستخدم المطورون الذكاء الاصطناعي لأداء المهام التي يتم تنفيذها يدويا بكفاءة أكبر والتواصل مع العملاء وتحديد الأنماط وحل المشكلات. يستخدم المطورون الذكاء الاصطناعي لأداء المهام التي يتم تنفيذها يدويا بكفاءة أكبر والتواصل مع العملاء وتحديد الأنماط وحل المشكلات. يجب أن يكون للمطورين خلفية في الرياضيات ويشعرون بالراحة مع الخوارزميات. التعلم عن طريق الممارسة وسيلة رائعة لتحسين أي مهارة.


### Example 2: Original article
لقد أصبح الذكاء الاصطناعي مصطلحًا شاملًا للتطبيقات التي تؤدي مهام مُعقدة كانت تتطلب في الماضي إدخالات بشرية مثل التواصل مع العملاء عبر الإنترنت أو ممارسة لعبة الشطرنج. يُستخدم غالبًا هذا المصطلح بالتبادل مع مجالاته الفرعية، والتي تشمل التعلم الآلي (ML) والتعلم العميق.

ومع ذلك، هناك اختلافات.. على سبيل المثال، يُركز التعلم الآلي على إنشاء أنظمة تتعلم أو تحسّن من أدائها استنادًا إلى البيانات التي تستهلكها. ومن المهم أن نلاحظ أنه على الرغم من أن كل سُبل التعلم الآلي ما هي إلّا ذكاء اصطناعي، فإنه ليس كل ذكاء اصطناعي يُعد تعلمًا آليًا.

للحصول على القيمة الكاملة من الذكاء الاصطناعي، تقوم العديد من الشركات باستثمارات كبيرة في فرق علوم البيانات. يجمع علم البيانات بين الإحصاءات وعلوم الكمبيوتر والمعرفة بالأعمال لاستخلاص القيمة من مصادر البيانات المختلفة.
يستخدم المطورون الذكاء الاصطناعي لأداء المهام التي يتم تنفيذها يدويًا بكفاءة أكبر، والتواصل مع العملاء، وتحديد الأنماط، وحل المشكلات. للبدء في استخدام الذكاء الاصطناعي، يجب أن يكون للمطورين خلفية في الرياضيات ويشعرون بالراحة مع الخوارزميات.

عند البدء باستخدام الذكاء الاصطناعي لإنشاء تطبيق، يساعد على البدء على نطاق صغير. من خلال بناء مشروع بسيط نسبيًا، مثل tic-tac-toe، على سبيل المثال، ستتعلم أساسيات الذكاء الاصطناعي. يعد التعلم عن طريق الممارسة وسيلة رائعة لتحسين أي مهارة، والذكاء الاصطناعي لا يختلف عن ذلك. بمجرد الانتهاء من مشروع صغير أو أكثر بنجاح، لا توجد حدود للمكان الذي يمكن أن يأخذك فيه الذكاء الاصطناعي.

#### Generated summary
 الذكاء الاصطناعي مصطلح شامل للتطبيقات التي تؤدي مهام مُعقدة كانت تتطلب إدخالات بشرية. يمكن استخدام هذا المصطلح بالتبادل مع مجالاته الفرعية وتشمل التعلم الآلي والتعلم العميق والتعلم العميق. يستخدم المطورون الذكاء الاصطناعي لأداء المهام التي يتم تنفيذها يدويا بكفاءة أكبر والتواصل مع العملاء وتحديد الأنماط وحل المشكلات. يستخدم المطورون الذكاء الاصطناعي لأداء المهام التي يتم تنفيذها يدويا بكفاءة أكبر والتواصل مع العملاء وتحديد الأنماط وحل المشكلات. يجب أن يكون للمطورين خلفية في الرياضيات ويشعرون بالراحة مع الخوارزميات. التعلم عن طريق الممارسة وسيلة رائعة لتحسين أي مهارة

### Calculating the rouge score

Method to compute the ROUGE sore to evalute the model

In [46]:
import torch
from tqdm import tqdm

model.eval()

# initializing progress bar 
progress_bar = tqdm(total=len(test_loader))

all_preds = []
all_labels = []

# Loop through the test_loader to generate predictions
for batch in test_loader:
    with torch.no_grad():
        # Get the input IDs and labels from the batch
        input_ids = batch['input_ids'].to(model.device)
        labels = batch['labels'].to(model.device)
        
        # Generate summaries/predictions
        preds = model.generate(input_ids, max_length=150, num_beams=2, length_penalty=1.2)

        # Collect predictions and labels
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        
    # Update progress bar
    progress_bar.update(1)

# Close the progress bar
progress_bar.close()

 29%|██▉       | 257/885 [4:55:16<11:32:02, 66.12s/it]

KeyboardInterrupt: 

The prediction was stopped around `30%` of the dataset so any metrics we shall calculate might be influenced by the fact that we didn't use all `13k` samples, However based on the performance of the model on out-of-sample web articles and compared to other trials of fine tuning it's performing quite well given the hardware and time limitations we're working with 

In [44]:
# import nltk
# import numpy as np
# import evaluate

# metric = evaluate.load("rouge")

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Rouge expects a newline after each sentence
#     decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
#     decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

#     # Note that other metrics may not have a `use_aggregator` parameter
#     # and thus will return a list, computing a metric for each sentence.
#     result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True, tokenizer=lambda x: x.split())
#     # Extract a few results
#     result = {key: value * 100 for key, value in result.items()}

#     # Add mean generated length
#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
#     result["gen_len"] = np.mean(prediction_lens)

#     return {k: round(v, 4) for k, v in result.items()}

In [75]:
def rouge_score(eval_pred):
    predictions, labels = eval_pred
    
    # Ensure predictions and labels are lists of arrays
    flat_preds = [pred.tolist() for pred in predictions]
    flat_labels = [label.tolist() for label in labels]

    # Replace -100 in predictions and labels with the padding token
    processed_preds = []
    for pred in flat_preds:
        processed_pred = np.where(np.array(pred) != -100, pred, tokenizer.pad_token_id)
        processed_preds.append(processed_pred)

    processed_labels = []
    for label in flat_labels:
        processed_label = np.where(np.array(label) != -100, label, tokenizer.pad_token_id)
        processed_labels.append(processed_label)

    # Decode the predictions and labels
    decoded_preds = tokenizer.batch_decode(processed_preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(processed_labels, skip_special_tokens=True)

    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(pred.strip() for pred in pred.split('.')) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip() for label in label.split('.')) for label in decoded_labels]

    # Compute ROUGE score
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # Format the result
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    return result

In [77]:
# Compute ROUGE score using the compute_metrics() function
eval_preds = (all_preds, all_labels)
metrics = rouge_score(eval_preds)

print("ROUGE Scores:", metrics)

ROUGE Scores: {'rouge1': 12.213894325036048, 'rouge2': 4.340387739657083, 'rougeL': 11.971243158961158, 'rougeLsum': 12.096626129851842}
